# Image, steering angle and x and y logging

In [3]:
import carla
import numpy as np
import os
from datetime import datetime
import cv2
import time

class DataCollector:
    def __init__(self, client=None, world=None):
        self.client = client or carla.Client('localhost', 2000)
        self.world = world or self.client.get_world()
        if 'Town04' not in self.world.get_map().name:
            self.world = self.client.load_world('Town04')
        self.session = self.create_session()
        
    def create_session(self):
        session_id = datetime.now().strftime('%Y%m%d_%H%M%S')
        session = {
            'id': session_id,
            'image_dir': f'data/images/{session_id}',
            'labels_file': f'data/labels/{session_id}.npy'
        }
        os.makedirs(session['image_dir'], exist_ok=True)
        os.makedirs(os.path.dirname(session['labels_file']), exist_ok=True)
        return session
        
    def setup_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        vehicle_bp = bp_lib.find('vehicle.tesla.model3')
        spawn_points = self.world.get_map().get_spawn_points()
        
        # Try multiple spawn points until success
        for spawn_point in spawn_points:
            self.vehicle = self.world.try_spawn_actor(vehicle_bp, spawn_point)
            if self.vehicle is not None:
                return self.vehicle
                
        raise RuntimeError("Failed to spawn vehicle - no valid spawn points")
        
    def setup_camera(self):
        camera_bp = self.world.get_blueprint_library().find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', '640')
        camera_bp.set_attribute('image_size_y', '480')
        camera_bp.set_attribute('fov', '110')
        
        camera_init_trans = carla.Transform(carla.Location(x=2.0, z=1.4))
        self.camera = self.world.spawn_actor(camera_bp, camera_init_trans, attach_to=self.vehicle)
        
        self.image_array = None
        self.camera.listen(lambda image: self._process_image(image))
        
    def _process_image(self, image):
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3]  # Remove alpha channel
        self.image_array = array
        
    def collect_data(self, num_frames=1000, interval=0.1):
        labels = np.zeros((num_frames, 3), dtype=np.float32)  # [steering, x, y]
        
        self.vehicle.set_autopilot(True)
        time.sleep(2)  # Let autopilot stabilize
        
        for i in range(num_frames):
            if self.image_array is not None:
                # Save image
                img_path = os.path.join(self.session['image_dir'], f'{i:06d}.jpg')
                print(f'Saving {img_path}')
                cv2.imwrite(img_path, self.image_array)
                
                # Get vehicle data
                control = self.vehicle.get_control()
                location = self.vehicle.get_location()
                
                # Store labels
                labels[i] = [control.steer, location.x, location.y]
                
                time.sleep(interval)
            
        np.save(self.session['labels_file'], labels)
        print(f'Saved labels to {self.session["labels_file"]}')
        
    def cleanup(self):
        if hasattr(self, 'camera'):
            self.camera.destroy()
        if hasattr(self, 'vehicle'):
            self.vehicle.destroy()

In [ ]:
client = carla.Client('localhost', 2000) 
world = client.get_world() 

collector = DataCollector(client, world)  # Using existing CARLA connection
collector.setup_vehicle()
collector.setup_camera()
collector.collect_data()


: 